# Haplotyper Overview

### Input
  - gene definition file
  - per-barcode variants file 
  
  
### For each allele
  - make a set containing the allelic variants
  - classify each variant as either known or novel
  - compare the set of allele variants with the variants for each defined haplotype
    - set of variants shared between allele and haplotype
    - fraction of haplotype variants observed in allele
    - Jaccard coefficient of the allele variants and haplotype variants
    - Number of 'significant' variants for the haplotype present in the allele sequence

### Filter and sort the results
  - ignore any haplotypes which do not have snps
  - sort by jaccard, fraction and number of 'significant' variants
    - best 'scoring' haplotypes will be at the head of the list

### Output
  - all matches and summary information in json format for further processing
  - a tabular summary for humans

In [59]:
import json, yaml

# load reference data
with open("../../../CYP2D6_Pipeline/test_data/gene/CYP2D6.yaml", "r") as infile:
    gene = yaml.safe_load(infile)
    
# make a set containing all reference variants for the gene
reference_variants = {gene["snps"][snp]["g_notation"] for snp in gene["snps"]}
       
def characterise_allele(found_variants, gene_definition):
    """
    Compare the variants for an allele with all the known haplotypes.
    Characterise the commonality between the allele and the reference haplotypes.
    
    :param found_variants: A set of variants observed in the allele sequence
    :type found_variants: {str, str, ...} where str is a string containing an hgvs variant description
    
    :param gene_definition: The parsed gene file containing snp and haplotype definitions
    :type gene_definition: dict
    
    :return: a list of dicts where each dict characterises the commonality between the allelic
             variants and the variants for a single haplotype
    """
    # list to hold the comparisons 
    summary = []
    
    # compare the allele variants with those for each defined haplotype
    for haplotype in (h for h in gene_definition["haplotypes"] if len(h["snps"]) > 0):
        # the variants for this haplotype 
        haplotype_variants = {gene_definition["snps"][s]["g_notation"] for s in haplotype["snps"]}
        
        # the variants shared between the allele and the haplotype
        shared_variants = found_variants & haplotype_variants
        
        # summarize
        try:
            summary.append(
                {
                    # the current haplotype
                    "haplotype": haplotype["type"], 
                    
                    # the variants for this haplotype
                    "haplotype_variants": sorted(list(haplotype_variants)), 
                    
                    # the variants shared between allele and haplotype
                    "shared_variants": sorted(list(shared_variants)), 
                    
                    # the fraction of haplotype variants observed
                    "fraction": len(shared_variants)/ len(haplotype_variants),
                    
                    # jaccard coefficient of haplotype and allele variants
                    "jaccard": len(shared_variants) / (len(haplotype_variants) + len(found_variants) - len(shared_variants)),
                   
                    # shared 'significant' variants
                    "significant": [
                        gene_definition["snps"][s]["g_notation"] for s in haplotype["snps"] if gene_definition["snps"][s]["tags"] is not None and \
                        "significant" in gene_definition["snps"][s]["tags"] and \
                        gene_definition["snps"][s]["g_notation"] in shared_variants
                    ]
                }
            )
        except ZeroDivisionError:
            pass
    
    return summary


def match_allele(variants, gene, trim_boundary=False):
    """
    Compare the allele variants against the haplotype definitions
    Return a dictionary summarizing the comparison
    """
    # sort the variants
    variants = sorted(variants)
    
    # trim the first and last variants if requested
    if trim_boundary:
        if len(variants) > 0 and "ins" in variants[0]:
            variants = variants[1:]
    
        if len(variants) > 0  and "ins" in variants[-1]:
            variants = variants[:-1]
    
    # set of variants found in the allele
    found_variants = set(variants)
    
    # known variants - intersection of found variants with the set of all reference variants
    known_variants = found_variants & reference_variants
    
    # novel variants found in allele but not found in the reference set
    novel_variants = found_variants - reference_variants
    
    # significant variants found in allele
    significant_variants = {
        gene["snps"][s]["g_notation"] for s in gene["snps"] if gene["snps"][s]["tags"] is not None and \
        "significant" in gene["snps"][s]["tags"] and \
        gene["snps"][s]["g_notation"] in found_variants
    }

    # get all haplotype matches for this allele
    characterised_allele = characterise_allele(found_variants, gene)
    
    # filter to remove matches with zero overlap
    filtered_allele = [x for x in characterised_allele if x["fraction"] > 0]
    
    # sort by number of significant variants, fraction and jaccard
    # this gives a list of haplotypes ordered by decreasing 'score'
    sorted_allele = sorted(filtered_allele, key=lambda x: (len(x["significant"]), x["fraction"], x["jaccard"]), reverse=True)
    
    return {
        "known_variants": sorted(list(known_variants)),
        "novel_variants": sorted(list(novel_variants)),
        "significant_variants": sorted(list(significant_variants)),
        "haplotypes":     sorted_allele
    }


def tabulate_allele_matches(allele, match, num_hits=None):
    """
    Convert allele match information into a tabular representation for printing
    """
    overview = "\n".join(
        [
            "- {}    %id: {}".format(allele["sequence_id"], allele["identity"]),
            "- Variants: {}    Known: {}    Novel: {}    Significant: {}".format(
                len(match["known_variants"]) + len(match["novel_variants"]),
                len(match["known_variants"]),
                len(match["novel_variants"]),
                len(match["significant_variants"])
            )
        ]
    )
    
    report_hits = match["haplotypes"][:num_hits]
    columns = []
    columns.append(["{:25}".format("Variant|Haplotype")] + \
                   ["\033[1m{:25}\033[0m".format(v) if v in match["significant_variants"] else "{:25}".format(v) for v in match["known_variants"]] + \
                   ["{:25}".format(label) for label in ["Fraction", "Proportion", "Jaccard", "Significant"]])
    
    for hit in report_hits:
        column = ["{:6}".format(hit["haplotype"])]
        column += ["{:6}".format("*" if v in hit["haplotype_variants"] else " ") for v in match["known_variants"]]
        column.append("{:6}".format("{}/{}".format(len(hit["shared_variants"]), len(hit["haplotype_variants"]))))
        column.append("{:6}".format(str(hit["fraction"])[:4]))
        column.append("{:6}".format(str(hit["jaccard"])[:4]))
        column.append("{:<6}".format(len(hit["significant"])))
        columns.append(column)
        
    results = []
    for row in range(len(columns[0])):
        results.append("".join([c[row] for c in columns]))
    
    table = "\n".join(results)
    
    return (overview, table)

In [64]:
# load barcode variants and display
with open("../../variant_calling/96_well_output/variants/lbc66.json", "r") as infile:
    alleles = json.load(infile)

results = []
for allele in alleles:
    match = match_allele(allele["variants"], gene)
    match["sequence_id"] = allele["sequence_id"]
    
    table = tabulate_allele_matches(allele, match, 25)
    
    results.append(
        (
            json.dumps(match, indent=4, separators=(',', ': ')),
            "\n".join(table)
        )
    )


for result in results:
    print(result[1])
    print()

- Barcodelbc66--lbc66_Cluster0_Phase1_NumReads245    %id: 0.9843298341700898
- Variants: 29    Known: 23    Novel: 6    Significant: 1
Variant|Haplotype        *35A  *35   *35XN *35B  *2A   *2    *2XN  *2D   *39   *34   *31   *56A  *105  *98   *104  *21B  *51   *88   *102  *41   *41X2 *2M   *14B  *21   *65   
42126310C>T                                *                                         *           *           *                       *                 *                 *     
42126611C>G              *     *     *     *     *     *     *     *     *           *     *     *     *     *     *     *     *     *     *     *     *     *     *     *     
42127001G>A                                *                                         *     *     *     *     *                 *     *                 *                 *     
42127207C>T                                *                                         *     *           *                       *                       *         

In [4]:
%%html
<style>.container { width:95% !important; }</style>